<a href="https://colab.research.google.com/github/Georg1986/spoc/blob/master/Delta_Microscope_Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Open Source Autofocus
This is the documentation for the implemented autofocus mechanism into the delta microscope

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov  6 09:49:03 2019

@author: Georg
"""

#autofocus
#programs used: Python 3.6.9, Pd-l2ork-2.9.0
#FlyCap2 2.13.3.61 driver and respective PyCapture2
#following files where used: 
#autofocusv5.py (this python code)
#191021_delta_module_cam_trigger_endstops.pd
#this is dependent on the file o.io.slipserial.pd
#191029_delta_module_OSC_Receiver-Georg.pd

# To use the following functions, you have to import:
import cv2
import PyCapture2
import numpy as np
import time
import datetime
import os

# It is advised to set a working directory, like this:
os.chdir(r'C:\SPOC\DOC\Calibration\images')

def startcam():
    bus = PyCapture2.BusManager()
    numCams = bus.getNumOfCameras()
    print(numCams)
    camera = PyCapture2.Camera()
    uid = bus.getCameraFromIndex(0)
    camera.connect(uid)
    camera.startCapture()

def variance_of_laplacian(image):
	# First compute the Laplacian of the image and then return the focus
	# measure, which is simply the variance of the Laplacian
    # Suggestion: Use Gaussian Blur before Laplacien.
	return cv2.Laplacian(image, cv2.CV_64F).var()

def takemicropic(i):
    image = camera.retrieveBuffer()
    row_bytes = float(len(image.getData())) / float(image.getRows())
    cv_image = np.array(image.getData(), dtype="uint8").reshape((image.getRows(), image.getCols()) )
    #cv2.imshow('frame',cv_image)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    filename = ("position%s_%s.jpg" % (i, timestr))
    cv2.imwrite(filename, img=cv_image)
    return filename


# This produces 40 pictures, named 1-4 with timestamp
# The estimated time for position change is 25 seconds
def takecalpics(cycles, positions, movement_time):
    c = 1
    while (c <= cycles):
        i = 0
        while (i <= positions):
            takemicropic(i)
            #time.sleep(movement_time) # Excluded for testing
            i += 1
        c += 1


def autofocus(iterations):
    # A maximum lap value is desired, a z position to that should be stored
    # This needs to be optimized, maybe hill climbing works well here?
    # https://www.geeksforgeeks.org/introduction-hill-climbing-artificial-intelligence/
    
    lap0 = 10 # Just a starting value
    lap1 = 20000 # Another(!) starting value
    direction = 1 # This value is given to movement(direction), 0 = down, 1 = up
    number = 0 # The picture number, this increases each time a new picture is taken   
    filename = takemicropic(number) #gives the number to take to takepic, gets filename back
    
    # This is converting the pic to grey for  analysis
    image = cv2.imread(filename) # Add ",0" to load pic in gray, if the picture is colored
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Alternatively, convert to grey
    lap0 = variance_of_laplacian(image) # This gives back a value that lies mostly between 0 and 10000
    
    #-------------------------------------------------------------------------------
    # This is about classes, so far its functionality is unused
    # It could be used to create an array of pictures,
    # with filename, laplacian-value and xyz coordinates
    # and possibly also with the picture itself.
    p0 = Picture(filename, lap0)
    print(p0.lap)
    #-------------------------------------------------------------------------------
    
    while(number <= iterations):  # A recursive "solution" to find a high laplacien value
        #filename = takepic(number) #for debug, take pic from webcam
        filename = takemicropic(number) #take a picture, store the filename
        image = cv2.imread(filename) #,0 to load pic in gray, if neccessary
        #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lap1 = variance_of_laplacian(image)

        # This writes the calculated Laplacien on the loaded image
        text = "Laplacian"
        cv2.putText(image, "{}: {:.2f}".format(text, lap1), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
        cv2.imwrite(filename, image)

        number += 1
        print(lap0)
        print(lap1)
        if lap0 >= lap1:
            direction^=True #if lap0 is bigger than lap1, change direction
            print("toggle")

        movement(direction) 
        lap0 = lap1
    #return(This could return the found z value, if implemented)

# The functions of this module should be imported where needed,
# or can be tested here:
#startcam()
#autofocus(10)
